In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [88]:
req = requests.get('https://www.selver.ee/')
soup = BS4(req.content, features='html.parser')
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')
price_and_qty = re.compile(r'\s+(\d+.+)')
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':url_class})
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96&p=" for i in find_tag]

In [89]:
html_list = list()
list_of_products = list()
list_of_prices = list()
dict_of_products = dict()

In [92]:
#start of the urls and end of the URLS
for z in range(5,23):
    #Find product last page
    product_html = requests.get(get_urls[z])
    #puhasta ilusa seebiga
    soup = BS4(product_html.content, features='html.parser')
    try:
        find_product_last_page = soup.find('a',{'class':'last'}).text
    except AttributeError:
        find_product_last_page = 3
    
    for i in range(1,int(find_product_last_page)+1):
        product_html = requests.get(get_urls[z]+str(i))
        soup = BS4(product_html.content, features='html.parser')
        html_list.append(soup)  
        print(f"Page URL: {get_urls[z]+str(i)} Last page is: {find_product_last_page}")

    for y in range(len(html_list)):  
        find_product_group = html_list[y].find_all('li',{'class':product_class})
        find_product_group_prices = html_list[y].find_all('span',{'class':'price'})
        find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
        prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]
        for t,f in zip(find_product_group_titles,prices):
            dict_of_products.update({t:re.split(price_and_qty,f)[:2]})
          


Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=1 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=2 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=3 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=4 Last page is: 4
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=1 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=2 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=3 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=4 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=5 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=6 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=7 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=8 Last page is: 9
Page URL: https://www.selver

Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=10 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=11 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=12 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=1 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=2 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=3 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=4 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=5 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=6 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=7 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=8 Last page is: 12
Page URL: https://www.selver.ee/majapid

In [96]:
from sqlalchemy import create_engine, insert, update, MetaData, Table, String, Column, Integer

In [97]:
table_conn = create_engine('sqlite:///selver_data.db', echo=True)
toode = [{'Toode':x,'Hind':y[0],'Hind/Kogus':y[1]} for x,y in dict_of_products.items()]

In [99]:
conn = table_conn.connect()
meta = MetaData(table_conn)
table = Table(
               'Selver', meta,
               Column('id', Integer, primary_key=True, autoincrement=True),
               Column('Toode', String),
               Column('Hind', String),
               Column('Hind/Kogus', String),
)
meta.create_all()

conn.execute(table.insert(),toode)


2021-03-04 18:07:15,251 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-04 18:07:15,255 INFO sqlalchemy.engine.base.Engine ()
2021-03-04 18:07:15,259 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-04 18:07:15,262 INFO sqlalchemy.engine.base.Engine ()
2021-03-04 18:07:15,270 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Selver")
2021-03-04 18:07:15,273 INFO sqlalchemy.engine.base.Engine ()
2021-03-04 18:07:15,276 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Selver")
2021-03-04 18:07:15,278 INFO sqlalchemy.engine.base.Engine ()
2021-03-04 18:07:15,280 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Selver" (
	id INTEGER NOT NULL, 
	"Toode" VARCHAR, 
	"Hind" VARCHAR, 
	"Hind/Kogus" VARCHAR, 
	PRIMARY KEY (id)
)


2021-03-04 18:07:15,282 INFO sqlalchemy.engine.base.Engine ()
2021-03-04 18:07:15,295 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-04 1

In [98]:
toode

[{'Toode': 'Banaan Chiquita, kg',
  'Hind': '1,39 €',
  'Hind/Kogus': '1,39 €/kg '},
 {'Toode': 'Kurk Luunja, kg', 'Hind': '5,99 €', 'Hind/Kogus': '5,99 €/kg '},
 {'Toode': 'Kirssploomtomat, 250 g',
  'Hind': '1,29 €',
  'Hind/Kogus': '5,16 €/kg '},
 {'Toode': 'Kirsskobartomat, 500 g',
  'Hind': '2,99 €',
  'Hind/Kogus': '5,98 €/kg '},
 {'Toode': 'Apelsin Navel, kg', 'Hind': '1,59 €', 'Hind/Kogus': '1,59 €/kg '},
 {'Toode': 'Frillis lõigatud, tk',
  'Hind': '1,99 €',
  'Hind/Kogus': '1,99 €/kg '},
 {'Toode': 'Porgand, LAHEOTSA, 500 g',
  'Hind': '1,29 €',
  'Hind/Kogus': '1,29 €/tk '},
 {'Toode': 'Õun Golden pakitud, 1 kg',
  'Hind': '0,59 €',
  'Hind/Kogus': '1,18 €/kg '},
 {'Toode': 'Kobartomat, kg', 'Hind': '1,29 €', 'Hind/Kogus': '1,29 €/kg '},
 {'Toode': 'Avokaado karbis Ready to Eat, 300 g',
  'Hind': '1,79 €',
  'Hind/Kogus': '1,79 €/kg '},
 {'Toode': 'Pirn Conference, 1 kg',
  'Hind': '2,99 €',
  'Hind/Kogus': '9,97 €/kg '},
 {'Toode': 'Õun Jonagold, kg', 'Hind': '2,49 €', 'Hin

In [91]:
test = re.compile(r'\s+(\d+.+)')
stringOne = '2,19 €  2,19 tk '
stringTwo = '1,39 € 1,39 €/kg'
prices = re.split(test,stringOne)
pricesTwo =  re.split(test,stringTwo)
print(prices)
print(pricesTwo)

['2,19 €', '2,19 tk ', '']
['1,39 €', '1,39 €/kg', '']
